# 4.0 EfficientNetB0 Model (Finetuned)

This Jupyter Notebook contains the code required to finetune the EfficientNetB0 model on the training dataset and evaluate its performance on the validation dataset. The notebook is divided into eight sections, each containing code for a specific task.

The notebook is structured into eight main sections:

1. **Importing Required Libraries:** This section imports the necessary libraries for the notebook, including TensorFlow, NumPy, Matplotlib, and Scikit-learn.

2. **Mapping Imagenet Class Index to Human-Readable Labels:** This section maps the ImageNet class index to human-readable labels, which is necessary for interpreting the results of the EfficientNetB0 model predictions.

3. **Preparing and Loading the Training and Validation Datasets:** This section prepares and loads the training and validation datasets for input to the model.

4. **Training the Model:** This section defines the EfficientNetB0 model architecture, compiles the model, and trains it on the training dataset.

5. **Plotting Training Log:** This section plots the training and validation accuracy and loss over the training epochs.

6. **Obtaining Predictions for Validation Set:** This section makes predictions on the validation dataset using the trained EfficientNetB0 model.

7. **Calculating Performance Metrics:** This section calculates performance metrics for the trained EfficientNetB0 model using Scikit-learn. It calculates the precision score, recall score, F1 score, and accuracy score for the predicted labels.

8. **Plotting Confusion Matrix:** This section plots the confusion matrix for the predicted and actual labels.

Furthermore, this notebook provides a comprehensive evaluation of the finetuned EfficientNetB0 model's performance on the validation dataset and provides insights into the model's strengths and weaknesses which will be compared to other models.

## 4.1 Importing Required Libraries

In [ ]:
import os
import json
import glob
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB0
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report, confusion_matrix


warnings.filterwarnings("ignore")

## 4.2 Mapping Imagenet Class Index to Human-Readable Labels

> The ImageNet metadata JSON file was obtained from: https://www.kaggle.com/keras/resnet50

In [ ]:
# Create an empty dictionary
imagenet2idx = {}

# Open the file 'imagenet_class_index.json' in read mode and assign its content to the dictionary
with open('imagenet_class_index.json') as f:
    idx2imagenet = json.load(f)
    
# Print the contents
idx2imagenet

In [ ]:
# Update the dictionary with keys and values from 'idx2imagenet'
# The keys are the first elements of the values in 'idx2imagenet' and the values are a list of the 
# corresponding key in 'idx2imagenet' and the second element of the values in 'idx2imagenet'
imagenet2idx = {v[0]: [k, v[1]] for k, v in idx2imagenet.items()}

# Print the contents
imagenet2idx

## 4.3 Preparing and Loading the Training and Validation Datasets

In [ ]:
# Define the file paths to the directories containing the training and validation image datasets
TRAIN_PATH = "imageset/train"
VAL_PATH = "imageset/val"

# Set the batch size, image size, and number of epochs for training the model
BATCH_SIZE = 16
IMG_SIZE = (224,224)
EPOCHS = 40

In [ ]:
# Load the image data into two `tf.data.Dataset` objects using `image_dataset_from_directory()`
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_PATH,
    labels='inferred',
    label_mode='int',
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_PATH,
    labels='inferred',
    label_mode='int',
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE)

In [ ]:
# Create a list of `class_names` by iterating through the `train_ds` `class_names` attribute
class_names = []
for class_name in train_ds.class_names:
    class_names.append(imagenet2idx[class_name][1])
class_names

In [ ]:
# Set the `AUTOTUNE` constant to automatically determine an optimal buffer size for `prefetch()`
AUTOTUNE = tf.data.AUTOTUNE

# Cache, shuffle, and prefetch the training and validation data for better performance
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 4.4 Training the Model

In [ ]:
# Define the input layer with shape
ip = tf.keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Create base model with EfficientNetB0 architecture, pretrained weights, and global average pooling
base_model = EfficientNetB0(include_top=False, input_tensor=ip, weights="imagenet", pooling='avg')(ip)

# Set base model to not be trainable
base_model.trainable = False

# Apply batch normalisation to output of base model
x = tf.keras.layers.BatchNormalization()(base_model)

# Apply dropout regularisation to output of batch normalisation
x = tf.keras.layers.Dropout(0.2)(x)

# Add final dense layer with softmax activation to output predicted class probabilities
op = tf.keras.layers.Dense(len(class_names), activation="softmax", name="pred")(x)

# Create the overall model by specifying the input and output layers
model = tf.keras.Model(ip, op)

# Compile model with Adam optimizer, sparse categorical cross-entropy loss, and accuracy metric
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Print summary of model architecture and number of parameters
model.summary()

In [ ]:
# Define a ModelCheckpoint callback to save the best model during training, based on the validation loss
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    "models/best_model_finetune.h5", 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto', 
    save_freq='epoch'
)

# Train the model using the training dataset for a specified number of epochs, with the validation dataset 
# used for evaluation
history = model.fit(
    train_ds, 
    epochs=EPOCHS, 
    validation_data=val_ds, 
    verbose=1, 
    callbacks=[checkpoint_callback]
)

## 4.5 Plotting Training Log

In [ ]:
# Set the default seaborn theme
sns.set_theme()

# Set the context for plotting
sns.set_context("paper")

# Create a new figure with a size of 9 inches x 5 inches
fig = plt.figure(figsize=(9,5))

# Add a subplot to the figure with a grid of 1 row and 2 columns, and select the first column
fig.add_subplot(1,2,1)

# Plot the training and validation accuracy over the epochs
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')

# Add a subplot to the figure with a grid of 1 row and 2 columns, and select the second column
fig.add_subplot(1,2,2)

# Plot the training and validation loss over the epochs
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')

# Adjust the subplot layout for better spacing
fig.tight_layout()

# Save the figure to a PNG file
plt.savefig("figures/model_performance_finetuned.png", dpi=300)

# Display the plot
plt.show()

## 4.6 Obtaining Predictions for Validation Set

In [ ]:
# Find all JPEG image files in the validation directory and store the paths in a list
val_images_list = glob.glob(os.path.join(VAL_PATH, "**", "*.JPEG"), recursive=True)

# Initialise empty lists for ground truth and predicted labels
actual_labels = []
predicted_labels = []

# Loop through each validation image path
for val_image_path in val_images_list:
    
    # Extract the ground truth label by finding the index of the class name
    # in a list of class names, using a dictionary to map from image file names to class names
    actual_labels.append(class_names.index(imagenet2idx[val_image_path.split("\\")[1]][1]))
    
    # Load the image, resize it to a target size, and convert it to a NumPy array
    img = image.load_img(val_image_path, target_size=IMG_SIZE)
    x = np.expand_dims(image.img_to_array(img), axis=0)

    # Predict class probabilities for input image and append predicted label index to list
    preds = model.predict(x, verbose=0)
    predicted_labels.append(np.argmax(preds))

In [ ]:
# Print ground truth labels
print("Ground truth labels:")
actual_labels

In [ ]:
# Print predicted labels
print("Predicted labels:")
predicted_labels

In [ ]:
# Print class name
print("Class names:")
class_names

## 4.7 Calculating Performance Metrics

In [ ]:
# Calculate precision score for the predicted labels
print("Precison:")
precision_score(actual_labels, predicted_labels, average='macro')

In [ ]:
# Calculate recall score for the predicted labels
print("Recall score:")
recall_score(actual_labels, predicted_labels, average='macro')

In [ ]:
# Calculate f1 score for the predicted labels
print("F1 score:")
f1_score(actual_labels, predicted_labels, average='macro')

In [ ]:
# Calculate accuracy score for the predicted labels
print("Accuracy score:")
accuracy_score(actual_labels, predicted_labels, normalize=True)

In [ ]:
# Print a classification report
print("Classification report:")
print(classification_report(actual_labels, predicted_labels, target_names=class_names))

## 4.8 Plotting Confusion Matrix

In [ ]:
# Set seaborn default theme
sns.set_theme()

# Set seaborn plot context
sns.set_context("paper")

# Create a new matplotlib figure object with size 7x5 inches
fig = plt.figure(figsize=(7,5))

# Calculate the confusion matrix for the predicted and actual labels
cm = confusion_matrix(actual_labels, predicted_labels)

# Create a pandas DataFrame from the confusion matrix
cm_df = pd.DataFrame(cm, index=[i for i in class_names], columns=[i for i in class_names])

# Create a heatmap of the confusion matrix using seaborn
sns.heatmap(cm_df, annot=True, fmt='g')

# Add a title to the plot
plt.title('Confusion Matrix for Val Split')  

# Adjust the layout of the plot
fig.tight_layout()  

# Save the figure to a PNG file
plt.savefig("figures/confusion_matrix_finetuned.png", dpi=300)  

# Display the plot
plt.show()

-------------------------------------------------------------------------------

#### Code adapted from:

https://keras.io/api/applications/

https://www.tensorflow.org/tutorials/images/classification

https://keras.io/examples/vision/image_classification_from_scratch/

https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/

https://www.kaggle.com/code/arjunrao2000/beginners-guide-efficientnet-with-keras/notebook